In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [73]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
268,tt0380510,65000000,93525586,The Lovely Bones,Rachel Weisz|Mark Wahlberg|Susan Sarandon|Saoi...,Peter Jackson,The story of a life and everything that came a...,"After being brutally murdered, 14-year-old Sus...",136,Fantasy|Drama,WingNut Films|DreamWorks SKG|Key Creatives|Fil...,12/26/2009,6.6,2009
1196,tt1606378,92000000,304654182,A Good Day to Die Hard,Bruce Willis|Jai Courtney|Sebastian Koch|Mary ...,John Moore,Yippee Ki-Yay Mother Russia,"Iconoclastic, take-no-prisoners cop John McCla...",98,Action|Thriller,Ingenious Media|Twentieth Century Fox Film Cor...,2/6/2013,5.2,2013
984,tt1667889,95000000,877244782,Ice Age: Continental Drift,John Leguizamo|Ray Romano|Chris Wedge|Denis Le...,Steve Martino|Mike Thurmeier,"Manny, Diego, and Sid embark upon another adve...","Manny, Diego, and Sid embark upon another adve...",88,Animation|Comedy|Adventure|Family,Blue Sky Studios|Twentieth Century Fox Animation,6/26/2012,6.2,2012
18,tt1661199,95000000,542351353,Cinderella,Lily James|Cate Blanchett|Richard Madden|Helen...,Kenneth Branagh,Midnight is just the beginning.,"When her father unexpectedly passes away, youn...",112,Romance|Fantasy|Family|Drama,Walt Disney Pictures|Genre Films|Beagle Pug Fi...,3/12/2015,6.8,2015
44,tt3622592,12000000,85512300,Paper Towns,Nat Wolff|Cara Delevingne|Halston Sage|Justice...,Jake Schreier,Get Lost. Get Found.,Quentin Jacobsen has spent a lifetime loving t...,109,Drama|Mystery|Romance,Fox 2000 Pictures,7/9/2015,6.2,2015


In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [15]:
# Предобработка
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


In [74]:
answers = {} # создадим словарь для ответов


# тут другие ваши предобработки колонок например:
def split_str(cast : str) -> str:
    return [actor for actor in cast.split('|')]
def is_profit(row):
    if row['budget'] < row.revenue:
        return True
    else:
        return False

data.cast = data.cast.apply(split_str)
data.production_companies = data.production_companies.apply(split_str)
data.genres = data.genres.apply(split_str)
#Привести к типу Datetime
data.release_date = pd.to_datetime(data.release_date)
# Добавить колонку окупаемости
data['is_profit'] = data.apply(is_profit, axis = 1)
data['profit'] = data.apply(lambda x: x['revenue'] - x['budget'], axis = 1)
data.head(5)
#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,is_profit,profit
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]","[Universal Studios, Amblin Entertainment, Lege...",2015-06-09,6.5,2015,True,1363528810
1,tt1392190,150000000,378436354,Mad Max: Fury Road,"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...",George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,"[Action, Adventure, Science Fiction, Thriller]","[Village Roadshow Pictures, Kennedy Miller Pro...",2015-05-13,7.1,2015,True,228436354
2,tt2908446,110000000,295238201,Insurgent,"[Shailene Woodley, Theo James, Kate Winslet, A...",Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,"[Adventure, Science Fiction, Thriller]","[Summit Entertainment, Mandeville Films, Red W...",2015-03-18,6.3,2015,True,185238201
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad...",J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,"[Action, Adventure, Science Fiction, Fantasy]","[Lucasfilm, Truenorth Productions, Bad Robot]",2015-12-15,7.5,2015,True,1868178225
4,tt2820852,190000000,1506249360,Furious 7,"[Vin Diesel, Paul Walker, Jason Statham, Miche...",James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,"[Action, Crime, Thriller]","[Universal Pictures, Original Film, Media Righ...",2015-04-01,7.3,2015,True,1316249360


# 1. У какого фильма из списка самый большой бюджет?

In [43]:
# посчитать пропущенные значения
data.isna().sum()

imdb_id                 0
budget                  0
revenue                 0
original_title          0
cast                    0
director                0
tagline                 0
overview                0
runtime                 0
genres                  0
production_companies    0
release_date            0
vote_average            0
release_year            0
is_paid_off             0
dtype: int64

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [56]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа +
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)' 
# если ответили верно, можете добавить комментарий со значком "+"

In [51]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()][['imdb_id','budget','original_title']]

,imdb_id,budget,original_title
723,tt1298650,380000000,Pirates of the Caribbean: On Stranger Tides


ВАРИАНТ 2

In [0]:
# можно добавлять разные варианты решения


# 2. Какой из фильмов самый длительный (в минутах)?

In [0]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals (tt0279111)'

In [54]:
data[data.runtime == data.runtime.max()][['imdb_id','budget','original_title','runtime']]

,imdb_id,budget,original_title,runtime
1157,tt0279111,56000000,Gods and Generals,214


In [148]:
# вариант второй
data.sort_values(by=['runtime'], ascending=False).head(1)[['imdb_id','budget','original_title','runtime']]


,imdb_id,budget,original_title,runtime
1157,tt0279111,56000000,Gods and Generals,214


# 3. Какой из фильмов самый короткий (в минутах)?





In [57]:
data[data.runtime == data.runtime.min()][['imdb_id','budget','original_title','runtime']]

,imdb_id,budget,original_title,runtime
768,tt1449283,30000000,Winnie the Pooh,63


In [149]:
# Второй вариант 
data.sort_values(by=['runtime']).head(1)[['imdb_id','budget','original_title','runtime']]

,imdb_id,budget,original_title,runtime
768,tt1449283,30000000,Winnie the Pooh,63


In [ ]:
# 4. Какова средняя длительность фильмов?


In [59]:
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [77]:
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [80]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data[(data.is_profit == True) & (data.profit == max(data.profit))][['imdb_id','budget','revenue','profit','original_title','runtime']]

,imdb_id,budget,revenue,profit,original_title,runtime
239,tt0499549,237000000,2781505847,2544505847,Avatar,162


# 7. Какой фильм самый убыточный? 

In [82]:
data[(data.is_profit == False) & (data.profit == min(data.profit))][['imdb_id','budget','revenue','profit','original_title','runtime']]

,imdb_id,budget,revenue,profit,original_title,runtime
1245,tt1210819,255000000,89289910,-165710090,The Lone Ranger,149


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [85]:
data[data.is_profit == True].is_profit.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [119]:
data.groupby(['release_year']).groups
max_revenue = data.groupby(['release_year'])['revenue'].max()
data[data.revenue == max_revenue.loc[2008]]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,is_profit,profit
599,tt0468569,185000000,1001921825,The Dark Knight,"[Christian Bale, Michael Caine, Heath Ledger, ...",Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,"[Drama, Action, Crime, Thriller]","[DC Comics, Legendary Pictures, Warner Bros., ...",2008-07-16,8.1,2008,True,816921825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [145]:
mins_profit = data.groupby(['release_year'])[['profit']].min()
mins_our_years = mins_profit.loc[[2012,2013,2014]].min()
mins_our_years[0]
data[data.profit == mins_our_years[0]]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,is_profit,profit
1245,tt1210819,255000000,89289910,The Lone Ranger,"[Johnny Depp, Armie Hammer, William Fichtner, ...",Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,"[Action, Adventure, Western]","[Walt Disney Pictures, Jerry Bruckheimer Films...",2013-07-03,6.0,2013,False,-165710090


# 11. Какого жанра фильмов больше всего?

In [0]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

# 14. Какой режисер снял больше всего фильмов в стиле Action?

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

# 18. Самый убыточный фильм от Paramount Pictures

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

# 20. Какой самый прибыльный год для студии Warner Bros?

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

# 23. Для какого режиссера зима – самое продуктивное время года? 

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

# Submission

In [0]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '...', '2': '...'}

In [0]:
# и убедиться что ни чего не пропустил)
len(answers)